## Attempt for Regression using GAT

In [2]:
import torch
from torch_geometric.nn import GATConv, global_mean_pool
import torch.nn.functional as F
import torch.nn as nn

In [3]:
class GAT(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers):
        super(GAT, self).__init__()
        # multilayer perceptron is used for encoding the features
        self.mlp = nn.Sequential(
        nn.Linear(input_dim, 32),
        nn.ReLU(),
        nn.Linear(32, 64))

        self.gat_layers = nn.ModleList([GATConv(hidden_dim, hidden_dim, heads=8, concat=True, dropout=0.2) for _ in range(num_layers-1)])
        self.final_gat = GATConv(hidden_dim*8, hidden_dim, heads=1, concat=False, dropout=0.2)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = self.mlp(x)

        for layer in self.gat_layers: 
            x_in = x
            x = F.dropout(x, p=0.2, training=self.training)
            x = F.elu(layer(x, edge_index))
            x = x + x_in 
        x = self.final_gat(x, edge_index)
        x = global_mean_pool(x, batch)
        x = self.fc(x) 
        return x